# Data Preprocessing

* Generate These Tables:
  * Merchant Order table
  * Merchant Product table
  * Time Series purchase, client table

In [91]:
import pandas as pd
from datetime import date, timedelta
import random
import numpy as np

## Understand Data

* Raw data is downloaded from
  * https://www.kaggle.com/c/instacart-market-basket-analysis/data
  * https://en.wikipedia.org/wiki/List_of_supermarket_chains_in_Canada

In [2]:
products_df = pd.read_csv('products.csv')
print(products_df.shape)

products_df.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [3]:
departments_df = pd.read_csv('departments.csv')
print(departments_df.shape)

departments_df.head()

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [4]:
aisles_df = pd.read_csv('aisles.csv')
print(aisles_df.shape)

aisles_df.head()

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [5]:
# order_dow is the day of the week
orders_df = pd.read_csv('orders.csv')
print(orders_df.shape)

orders_df.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
order_prior_df = pd.read_csv('order_products__prior.csv')
print(order_prior_df.shape)

order_prior_df.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
order_train_df = pd.read_csv('order_products__train.csv')
print(order_train_df.shape)

order_train_df.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


## Utils

In [76]:
def get_monday_lst(year):
    d = date(year, 2, 17)    
    d += timedelta(days = 0 - d.weekday())  # First Monday
    while d.year == year and d.month < 5:
        yield d
        d += timedelta(days = 7)

def uniform_random_select(lst, const):
    rand = random.randint(0, const%len(lst))
    return lst[rand]

## Merchant Order Table

In [62]:
# unit both prior and train orders
order_prior_train_df = order_prior_df.append(order_train_df)
print(order_prior_train_df.shape)

(33819106, 4)


In [64]:
order_prod_df = orders_df.merge(order_prior_train_df, on='order_id')
order_prod_df = order_prod_df.sort_values(by=['user_id', 'order_number'])
print(order_prod_df.shape)

order_prod_df.head()

(33819106, 10)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [117]:
monday_seeds = list(get_monday_lst(2020))

user_startingdate_dct = {}

def assign_date(r):
    user_id = r[1]
    days_since_prior_order = r[2]
    order_dow = r[3]
    
    if np.isnan(days_since_prior_order):
        row_date = uniform_random_select(monday_seeds, user_id) + timedelta(days = order_dow)
        user_startingdate_dct[user_id] = row_date
    else:
        row_date = user_startingdate_dct[user_id] + timedelta(days = days_since_prior_order)
        
    r = r.append(row_date)

In [97]:
user_group_df = order_prod_df[['order_id', 'user_id', 'days_since_prior_order', 'order_dow', 'order_number']].drop_duplicates()
user_group_df = user_group_df.sort_values(by=['user_id', 'order_number'])

print(user_group_df.shape)

user_group_df.head()

(3346083, 5)


,order_id,user_id,days_since_prior_order,order_dow,order_number
0,2539329,1,NaN,2,1
5,2398795,1,15.0,3,2
11,473747,1,21.0,3,3
16,2254736,1,29.0,4,4
21,431534,1,28.0,4,5


In [124]:
# Using a list is faster
user_group_records = user_group_df.values.tolist()

for r in user_group_records:
    assign_date(r)
    
print (user_group_records[0:4])

[[2539329.0, 1.0, nan, 2.0, 1.0, datetime.date(2020, 2, 26)], [2398795.0, 1.0, 15.0, 3.0, 2.0, datetime.date(2020, 3, 12)], [473747.0, 1.0, 21.0, 3.0, 3.0, datetime.date(2020, 3, 18)], [2254736.0, 1.0, 29.0, 4.0, 4.0, datetime.date(2020, 3, 26)]]


In [125]:
user_group_df = pd.DataFrame(user_group_records)
user_group_df.columns = ['order_id', 'user_id', 'days_since_prior_order', 'order_dow', 'order_number', 'purchase_date']
user_group_df.head()

,order_id,user_id,days_since_prior_order,order_dow,order_number,purchase_date
0,2539329.0,1.0,NaN,2.0,1.0,2020-02-26
1,2398795.0,1.0,15.0,3.0,2.0,2020-03-12
2,473747.0,1.0,21.0,3.0,3.0,2020-03-18
3,2254736.0,1.0,29.0,4.0,4.0,2020-03-26
4,431534.0,1.0,28.0,4.0,5.0,2020-03-25


In [127]:
order_prod_df = order_prod_df.merge(user_group_df[['order_id', 'purchase_date']], on='order_id')
print(order_prod_df.shape)

order_prod_df.head()

(33819106, 11)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,purchase_date
0,2539329,1,prior,1,2,8,NaN,196,1,0,2020-02-26
1,2539329,1,prior,1,2,8,NaN,14084,2,0,2020-02-26
2,2539329,1,prior,1,2,8,NaN,12427,3,0,2020-02-26
3,2539329,1,prior,1,2,8,NaN,26088,4,0,2020-02-26
4,2539329,1,prior,1,2,8,NaN,26405,5,0,2020-02-26
